# Using Ray for Highly Parallelizable Tasks

While Ray can be used for very complex parallelization tasks,
often we just want to do something simple in parallel.
For example, we may have 100,000 time series to process with exactly the same algorithm,
and each one takes a minute of processing.

Clearly running it on a single processor is prohibitive: this would take 70 days.
Even if we managed to use 8 processors on a single machine,
that would bring it down to 9 days. But if we can use 8 machines, each with 16 cores,
it can be done in about 12 hours.

How can we use Ray for these types of task? 

## Let's try to compute the value of PI


We take the simple example of computing the digits of pi using the [monte carlo](https://www.geeksforgeeks.org/estimating-value-pi-using-monte-carlo/) approach.
The algorithm is simple: generate random x and y, and if ``x^2 + y^2 < 1``, it's
inside the circle, we count as in. This actually turns out to be pi/4
(remembering your high school math).

<img src="https://miro.medium.com/max/770/0*Oaabaj-WXhI0Hu-T" width="70%" height="30%">

The following code (and this notebook) assumes you have already set up your Ray cluster and that you are running on the head node. For more details on how to set up a Ray cluster please see the [Ray Cluster Quickstart Guide](https://docs.ray.io/en/master/cluster/quickstart.html). 

📖 [Back to Table of Contents](../ex_00_tutorial_overview.ipynb)<br>



In [1]:
import os
import time
import logging
from pprint import pprint

import ray
import random
import math
from fractions import Fraction

In [2]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8267


We use the ``@ray.remote`` decorator to create a Ray task.
A task is like a function, except the result is returned asynchronously.

It also may not run on the local machine, it may run elsewhere in the cluster.
This way you can run multiple tasks in parallel,
beyond the limit of the number of processors you can have in a single machine.

In [3]:
@ray.remote
def pi4_sample(sample_count):
    """pi4_sample runs sample_count experiments, and returns the 
    fraction of time it was inside the circle. 
    """
    in_count = 0
    for i in range(sample_count):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            in_count += 1
    return Fraction(in_count, sample_count)


To get the result of a future, we use `ray.get()` which 
blocks until the result is complete. 

In [4]:
SAMPLE_COUNT = 1000 * 1000
start = time.time() 
future = pi4_sample.remote(sample_count = SAMPLE_COUNT)
pi4 = ray.get(future)
end = time.time()
dur = end - start
print(f'Running {SAMPLE_COUNT} tests took {dur} seconds')

Running 1000000 tests took 0.22035717964172363 seconds


Now let's see how good our approximation is.

In [5]:
pi = pi4 * 4

In [6]:
print(f" Our computed PI: {float(pi)}; real PI: {math.pi}")

 Our computed PI: 3.138556; real PI: 3.141592653589793


In [7]:
abs(pi-math.pi)/pi

0.0009675320720080237

Meh. A little off -- that's barely 4 decimal places.
Why don't we do it a 100,000 times as much? Let's do 100 billion!
`FULL_SAMPLE_COUNT = 100 * 1000 * 1000 * 1000 # 100 billion samples!` may take a while. 

Let's try with `1, 2 and 3b` and observe each value.


In [8]:
FULL_SAMPLE_COUNT = 1 * 1000 * 1000 * 1000 # 1 billion samples! 
start = time.time()
BATCHES = int(FULL_SAMPLE_COUNT / SAMPLE_COUNT)
print(f'Doing {BATCHES} batches')
results = []
for _ in range(BATCHES):
    results.append(pi4_sample.remote(sample_count = SAMPLE_COUNT))
output = ray.get(results)
end = time.time()
dur = end - start
print(f'Running {SAMPLE_COUNT} tests took {dur} seconds')

Doing 1000 batches
Running 1000000 tests took 21.831931114196777 seconds


Notice that in the above, we generated a list with 100,000 futures.
Now all we do is have to do is wait for the result.

Depending on your ray cluster's size, this might take a few minutes.
But to give you some idea, if we were to do it on a single machine,
when we ran this, it took 0.4 seconds.

On a single core, that means we're looking at 0.4 * 100000 = about 11 hours. 

So now, rather than just a single core working on this, we have all our cores working on the task together (depending on the number cores on each worker node). 

In [9]:
pi = sum(output)*4/len(output)

In [10]:
float(pi)

3.14151778

In [11]:
float(abs(pi-math.pi)/pi)

2.3833571870816713e-05

Not bad at all -- we're off by a millionth. 

### Reference

1. [How to scale Python multiprocessing to a cluster with one line of code](https://medium.com/distributed-computing-with-ray/how-to-scale-python-multiprocessing-to-a-cluster-with-one-line-of-code-d19f242f60ff) 
2. [Using Ray for Highly Parallelizable Tasks](https://docs.ray.io/en/master/ray-core/examples/highly_parallel.html)

📖 [Back to Table of Contents](../ex_00_tutorial_overview.ipynb)<br>